**Table of contents**<a id='toc0_'></a>    
- [ロボットアーム](#toc1_)    
- [pybulletの起動](#toc2_)    
- [pybulletの初期設定](#toc3_)    
- [ロボットアームの生成](#toc4_)    
- [シミュレーションの実行](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[ロボットアーム](#toc0_)

本notebookでは2軸のロボットアームを生成し、制御する手順について解説します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[pybulletの初期設定](#toc0_)

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
pybullet.setTimeStep(time_step) # 1stepあたりに経過する時間の設定

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[ロボットアームの生成](#toc0_)
今回は、2軸ロボットアーム`simple_2d_arm.urdf`を生成します。  
ロボットは下図のような構成になっています（センサーについては、`robot_arm_sensor.ipynb`にて解説しています。）  

![](../images/RobotArm/2d_robot_arm.png)

In [3]:
# ロボットの読み込み
arm_start_pos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
arm_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf",arm_start_pos, arm_start_orientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
camera_distance = 1.5
camera_yaw = 180.0 # deg
camera_pitch = -10 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[シミュレーションの実行](#toc0_)
`setJointMotorControl2` では、指定したロボットの指定したジョイントを動作させることができます。

最初に、速度指令で動作させるため、第三引数に`POSITION_CONTROL`を指定します。


In [4]:
import time
import math

# ロボットの各関節のインデックス
LINK1_JOINT_IDX = 0
LINK2_JOINT_IDX = 1

# 各関節の角度
joint1_angle = 90.0
joint2_angle = 90.0
joint1_rad = math.radians(joint1_angle)
joint2_rad = math.radians(joint2_angle)

for i in range(200):
    # 指定した角度に関節を動かす
    pybullet.setJointMotorControl2(arm_id, LINK1_JOINT_IDX, pybullet.POSITION_CONTROL, targetPosition=joint1_rad)
    pybullet.setJointMotorControl2(arm_id, LINK2_JOINT_IDX, pybullet.POSITION_CONTROL, targetPosition=joint2_rad)
    pybullet.stepSimulation()
    time.sleep(time_step)

次に、速度指令で動作させるため、第三引数に`VELOCITY_CONTROL`を指定します。

In [5]:
joint1_velocity = 2.0
joint2_velocity = 2.0

for i in range(500):
    # 指定した速度で関節を動かす
    pybullet.setJointMotorControl2(arm_id, LINK1_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=joint1_velocity)
    pybullet.setJointMotorControl2(arm_id, LINK2_JOINT_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=joint2_velocity)
    pybullet.stepSimulation()
    time.sleep(time_step)


最後に、トルク制御で動作させるため、第三引数に`TORQUE_CONTROL`を指定します。

In [6]:
joint1_torque = 50.0
joint2_torque = 50.0

for i in range(500):
    # 指定したトルクで関節を動かす
    pybullet.setJointMotorControl2(arm_id, LINK1_JOINT_IDX, pybullet.TORQUE_CONTROL, force=joint1_torque)
    pybullet.setJointMotorControl2(arm_id, LINK2_JOINT_IDX, pybullet.TORQUE_CONTROL, force=joint2_torque)
    pybullet.stepSimulation()
    time.sleep(time_step)